# Model Selection

In [2]:
import pandas as pd
import numpy as np
import glob, sys, os
sys.path.append('..')

/home/joel/anaconda3/envs/mds/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
from modules.plotting_metrics import PlotMetric
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='white', context='talk', font_scale=0.8)

/home/joel/anaconda3/envs/mds/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
file_name = './df_DkSc_results_COCRYS_CSAR_DEKOIS_DUD.pkl' # Created in 3_Calculating metrics...
X_merged_dksc = pd.read_pickle(file_name)
# Extract activity column
y_true_merged = X_merged_dksc['activity']
# Drop column from merged_dkksc
X_merged_dksc = X_merged_dksc.drop('activity', axis=1)
X_merged_dksc.shape


(3466, 402)

In [5]:
np.geomspace(1e-8, 1e4, 7)

array([1.e-08, 1.e-06, 1.e-04, 1.e-02, 1.e+00, 1.e+02, 1.e+04])

### Timeout Decorator

In [6]:
import signal
from functools import wraps

def timeout(n_seconds=300):
    '''Stops a function execution after n seconds'''
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Set alarm for n seconds
            signal.alarm(n_seconds)
            try:
                # Call decorated func
                return func(*args, **kwargs)
#             except TimeoutError as e:
                print(f'Execution finished after {n_seconds}:', e)
            finally:
                # Cancel Alarm
                signal.alarm(0)
        return wrapper
    return decorator

## Scaffold Splitting

In [7]:
#*************************************************
# Functions to compute stratify scaffold splitting
#*************************************************
sys.path.append('../2_Docking_analysis/')
from scaffold_splitter import train_test_scaffold_split

In [8]:
# Compute or load the dataframe containing the Generic Murcko Scaffolds
file = '../2_Docking_analysis/df_COCRYS_CSAR_DUD_DEKOIS_Murcko_Scaffolds_SMILES.obj'

df_scff_murcko = pd.read_pickle(file)
df_scff_murcko.shape

(3466, 3)

## Train/test on the same dataset 

### Learning Curves

In [9]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import learning_curve

In [10]:
def plot_learning_curves(estimator, X, y, title, ylim=[0.5,1], axes=None,
                         cv=3, train_sizes=np.linspace(0.1, 1.0, 10), 
                         scoring='roc_auc', n_jobs=4):
    '''
    Plot estimator performance on the training and validation
    sets as a function of the training set size.

    Parameters
    ----------
    estimator: sklearn estimator object type
       Object type that implements the "fit" and "predict method"

    X: array-like, shape (m_samples, n_features)
        Training array with m_samples and n_features.
    y: array-like, shape (m_samples)
        Target array relative to X with m labels.
    axes: array of 3 axes
        matplotlib axes array to append the generated plot
    ylim: array
       
    '''
    if axes == None:
        _, axes = plt.subplots(1, 1, figsize=(5, 5))
        
    axes.set(title=title, ylim=ylim, xlabel='Training examples', ylabel=f'Metric: {scoring}')
    # Use learning_curve function from sklearn
    train_sizes, train_scores, test_scores, fit_times, _ = \
       learning_curve(estimator, X, y, scoring=scoring, return_times = True,
                      cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    # Compute useful metrics
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
#     fit_times_mean = np.mean(fit_times, axis=1)
#     fit_times_std = np.std(fit_times, axis=1)
    # Plot the learning curve
    axes.grid()
    axes.fill_between(train_sizes, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1, color='r')
    axes.fill_between(train_sizes, test_scores_mean - test_scores_std,
                        test_scores_mean + test_scores_std, alpha=0.1, color='g')
    axes.plot(train_sizes, train_scores_mean, 'o-', color='r',
                label='Train score')
    axes.plot(train_sizes, test_scores_mean, 'o-', color='g',
                label='Cross-validation score')
    axes.legend(loc='lower right')
    

### Hyperparameters tunning: Grid Search

In [23]:
def run_grid_search(estimator, X_train, y_train, X_test, y_test, hyperparams,  cv_value=5, 
                    scoring='roc_auc', splitting='random', std_scale=False,
                   randomGS = False, n_iter=10):
    # Format the hyperparms
    hyperparams_dict = {'estimator__' + key: val for key, val in hyperparams.items()}
    
    if std_scale:
        # Create the Pipe
        scaler = StandardScaler()
        pipe = Pipeline([('scaler', scaler),
                         ('estimator', estimator)])
    else:
        pipe = estimator
    
    # Do Grid Search
    if randomGS:
        gs = RandomizedSearchCV(estimator = pipe,  param_distributions = hyperparams,
                     cv = cv_value, scoring = scoring, n_jobs = 6, refit = True, n_iter=n_iter)
    else:
        gs = GridSearchCV(estimator = pipe, param_grid = hyperparams,
                     cv = cv_value, scoring = scoring, n_jobs = 6, refit = True)
    # Train the model
    gs.fit(X_train, y_train) 

    # Predictions
    y_train_predict = gs.predict_proba(X_train)
    y_test_predict = gs.predict_proba(X_test)
    
    # Values to return
    n_train_mols    = y_train.shape[0]
    n_train_actives = y_train.sum()
    n_test_mols     = y_test.shape[0]
    n_test_actives  = y_test.sum()
    mean_cv_roc     = gs.best_score_
    train_roc       = roc_auc_score(y_train, y_train_predict[:, 1])
    test_roc        = roc_auc_score(y_test, y_test_predict[:, 1])
    best_params     = gs.best_params_

    # Print some values
    print(f'No. of molecules in train set: {n_train_mols}, with {n_train_actives} actives.')
    print(f'No. of molecules in test set: {n_test_mols}, with {n_test_actives} actives.')
    print('')
    print('*'*10, 'GRID SEARCH RESULTS', '*'*10)
    print('- Mean CV ROC-AUC:\t{:.3f}'.format(mean_cv_roc))
    print('- Train ROC-AUC:  \t{:.3f}'.format(train_roc))
    print('- Test ROC-AUC:   \t{:.3f}'.format(test_roc))
    print('- Best hyperparameters', best_params)
    print('**'*21)
    print('')
    
    return [n_train_mols, n_train_actives, n_test_mols, n_test_actives,
            mean_cv_roc, train_roc, test_roc, best_params]

### Function to report the Best Conformation's ROC-AUC for a given subset of samples

In [12]:
#************************************************************************
# Returns the best conformatio's ROC-AUC value of a given subset X and y
#************************************************************************

def get_roc_auc_DkSc(X_train, y_train, X_test, y_test, verbose=True):
    roc_auc_train = X_train.apply(
        lambda x: roc_auc_score(y_true= y_train, y_score= -x), axis=0)
    roc_auc_test = X_test.apply(
        lambda x: roc_auc_score(y_true= y_test, y_score= -x), axis=0)
    # Values to return
    train_best_roc = roc_auc_train.max()
    train_median   = roc_auc_train.median()
    train_mean     = roc_auc_train.mean()
    test_best_roc = roc_auc_test.max()
    test_median   = roc_auc_test.median()
    test_mean     = roc_auc_test.mean()
    
    if verbose:
        print("***** Best Conformation's ROC-AUC using docking scores *****")
        
        print("> Train best conf. ROC-AUC: {:.3f}".format(train_best_roc) +
              " \t> median: {:.3f}".format(train_median) +
              ', mean: {:.3f}'.format(train_mean))
        
        print("> Test best conf. ROC-AUC: {:.3f}".format(test_best_roc) +
              " \t> median: {:.3f}".format(test_median) +
              ', mean: {:.3f}'.format(test_mean))
        print('**'*32)
    # return a list of results to capture by the  wrapper function
    return [train_best_roc, train_median, train_mean,
            test_best_roc, test_median, test_mean]
        

In [13]:
#******************************************
# Decorator functions to capture GS results
#******************************************
from functools import wraps

def capture_GS_results(results_dict=None, capture=True):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if results_dict != None and capture:
                results = func(*args, **kwargs)
                # Create a key with the first four values
                key = '_'.join(results[:4])
                # Append results to the results dictionary
                results_dict[key] = results
            else:
                return func(*args, **kwargs)
        return wrapper
    return decorator
            
#******************************************************
# Dictionary to capture GS results using the decorator 
#******************************************************
results_dict = {}

#******************************************************
# Function to Split and run Grid Search
#******************************************************
@capture_GS_results(results_dict)
def split_and_gs(train_name, test_name, estimator_name,
                 X, y, estimator, hyperparams, splitting='random', 
                 test_size=0.25, scaffold_series=None, **kwargs):
    '''Given a X and y sets, a sklean estimator and an splitting method, 
    performs Grid Search CV using the parsed hyperparams'''
    #**************
    # Do the split
    #**************
    if splitting == 'scaffold':
        X_train, X_test, y_train, y_test = \
            train_test_scaffold_split(X, y, scaffold_series = scaffold_series,
                test_size=test_size, stratify=y)
    elif splitting == 'random':
        X_train, X_test, y_train, y_test = \
            train_test_split(X, y, test_size=test_size, stratify=y)
        
    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

In [14]:
results_dict

{}

#  Hyperparameter Tunning: Grid Search
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DEKOIS Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [15]:
#### library = 'DEKOIS'
library = 'DEKOIS'

# Train and test over DEKOIS
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [16]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 929, with 30 actives.
No. of molecules in test set: 310, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.858
- Train ROC-AUC:  	0.993
- Test ROC-AUC:   	0.838
- Best hyperparameters {'C': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.822 	> median: 0.708, mean: 0.707
> Test best conf. ROC-AUC: 0.835 	> median: 0.649, mean: 0.657
****************************************************************
CPU times: user 1.71 s, sys: 311 ms, total: 2.02 s
Wall time: 8.18 s


In [18]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
              X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 899, with 30 actives.
No. of molecules in test set: 340, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.822
- Train ROC-AUC:  	0.968
- Test ROC-AUC:   	0.839
- Best hyperparameters {'C': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.829 	> median: 0.733, mean: 0.732
> Test best conf. ROC-AUC: 0.808 	> median: 0.596, mean: 0.596
****************************************************************
CPU times: user 1.41 s, sys: 36 ms, total: 1.44 s
Wall time: 3.87 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [19]:
%%time
estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 929, with 30 actives.
No. of molecules in test set: 310, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.849
- Train ROC-AUC:  	0.976
- Test ROC-AUC:   	0.838
- Best hyperparameters {'C': 100.0, 'gamma': 1e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.814 	> median: 0.707, mean: 0.704
> Test best conf. ROC-AUC: 0.815 	> median: 0.665, mean: 0.668
****************************************************************
CPU times: user 3.01 s, sys: 126 ms, total: 3.14 s
Wall time: 28.4 s


In [20]:
%%time
estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 899, with 30 actives.
No. of molecules in test set: 340, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.828
- Train ROC-AUC:  	0.956
- Test ROC-AUC:   	0.873
- Best hyperparameters {'C': 100.0, 'gamma': 1e-08}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.829 	> median: 0.733, mean: 0.732
> Test best conf. ROC-AUC: 0.808 	> median: 0.596, mean: 0.596
****************************************************************
CPU times: user 2.69 s, sys: 165 ms, total: 2.86 s
Wall time: 27 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [21]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 929, with 30 actives.
No. of molecules in test set: 310, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.871
- Train ROC-AUC:  	0.960
- Test ROC-AUC:   	0.836
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.809 	> median: 0.688, mean: 0.688
> Test best conf. ROC-AUC: 0.829 	> median: 0.718, mean: 0.717
****************************************************************
CPU times: user 1.85 s, sys: 758 ms, total: 2.61 s
Wall time: 7.86 s


In [22]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C': np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 899, with 30 actives.
No. of molecules in test set: 340, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.832
- Train ROC-AUC:  	0.935
- Test ROC-AUC:   	0.825
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.829 	> median: 0.733, mean: 0.732
> Test best conf. ROC-AUC: 0.808 	> median: 0.596, mean: 0.596
****************************************************************
CPU times: user 2.01 s, sys: 653 ms, total: 2.67 s
Wall time: 7.6 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [23]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

kNN => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 929, with 30 actives.
No. of molecules in test set: 310, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.772
- Train ROC-AUC:  	0.771
- Test ROC-AUC:   	0.732
- Best hyperparameters {'n_neighbors': 225, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.778 	> median: 0.692, mean: 0.693
> Test best conf. ROC-AUC: 0.883 	> median: 0.702, mean: 0.702
****************************************************************
CPU times: user 1.92 s, sys: 104 ms, total: 2.03 s
Wall time: 3.81 s


In [24]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

kNN => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 899, with 30 actives.
No. of molecules in test set: 340, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.812
- Train ROC-AUC:  	0.828
- Test ROC-AUC:   	0.636
- Best hyperparameters {'n_neighbors': 125, 'p': 2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.829 	> median: 0.733, mean: 0.732
> Test best conf. ROC-AUC: 0.808 	> median: 0.596, mean: 0.596
****************************************************************
CPU times: user 1.88 s, sys: 65.6 ms, total: 1.94 s
Wall time: 3.51 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [25]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 929, with 30 actives.
No. of molecules in test set: 310, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.707
- Train ROC-AUC:  	0.761
- Test ROC-AUC:   	0.712
- Best hyperparameters {'criterion': 'gini', 'max_depth': 3, 'max_features': 'sqrt', 'min_samples_split': 0.3}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.800 	> median: 0.687, mean: 0.687
> Test best conf. ROC-AUC: 0.895 	> median: 0.722, mean: 0.726
****************************************************************
CPU times: user 1.85 s, sys: 104 ms, total: 1.96 s
Wall time: 3.1 s


In [26]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 899, with 30 actives.
No. of molecules in test set: 340, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.743
- Train ROC-AUC:  	0.912
- Test ROC-AUC:   	0.647
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_split': 0.25}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.829 	> median: 0.733, mean: 0.732
> Test best conf. ROC-AUC: 0.808 	> median: 0.596, mean: 0.596
****************************************************************
CPU times: user 1.77 s, sys: 98.8 ms, total: 1.87 s
Wall time: 2.84 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [27]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 25.8 ms, sys: 3.19 ms, total: 29 ms
Wall time: 25.9 ms


In [28]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 44 µs, sys: 6 µs, total: 50 µs
Wall time: 58.9 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DEKOIS  - Random and Stratified Scaffold Splitting</b>

In [29]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: DEKOIS; Test: DEKOIS; split: random
No. of molecules in train set: 929, with 30 actives.
No. of molecules in test set: 310, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.740
- Train ROC-AUC:  	0.817
- Test ROC-AUC:   	0.812
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.2, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.773 	> median: 0.682, mean: 0.682
> Test best conf. ROC-AUC: 0.886 	> median: 0.746, mean: 0.737
****************************************************************
CPU times: user 2.6 s, sys: 125 ms, total: 2.72 s
Wall time: 22 s


In [30]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams, 
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: DEKOIS; Test: DEKOIS; split: scaffold
No. of molecules in train set: 899, with 30 actives.
No. of molecules in test set: 340, with 10 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.783
- Train ROC-AUC:  	0.841
- Test ROC-AUC:   	0.662
- Best hyperparameters {'max_depth': 2, 'max_features': 'log2', 'min_samples_leaf': 0.2, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.829 	> median: 0.733, mean: 0.732
> Test best conf. ROC-AUC: 0.808 	> median: 0.596, mean: 0.596
****************************************************************
CPU times: user 2.13 s, sys: 45.6 ms, total: 2.18 s
Wall time: 23.5 s


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with DUD Library
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

In [31]:
library = 'DUD'

# Train and test over DUDU
X = X_merged_dksc.loc[library]
y = y_true_merged.loc[library]
scaffold_series = df_scff_murcko['scff_generic'].loc[library]

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [32]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 1368, with 43 actives.
No. of molecules in test set: 457, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.962
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.890
- Best hyperparameters {'C': 1.0}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.693 	> median: 0.599, mean: 0.581
> Test best conf. ROC-AUC: 0.653 	> median: 0.535, mean: 0.533
****************************************************************
CPU times: user 2 s, sys: 66.7 ms, total: 2.07 s
Wall time: 7.69 s


In [33]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 1325, with 43 actives.
No. of molecules in test set: 500, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.908
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.894
- Best hyperparameters {'C': 1.0}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.644 	> median: 0.529, mean: 0.517
> Test best conf. ROC-AUC: 0.847 	> median: 0.739, mean: 0.720
****************************************************************
CPU times: user 1.85 s, sys: 52.2 ms, total: 1.91 s
Wall time: 7.14 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [34]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'gamma':  np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 1368, with 43 actives.
No. of molecules in test set: 457, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.954
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.981
- Best hyperparameters {'C': 100.0, 'gamma': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.660 	> median: 0.552, mean: 0.538
> Test best conf. ROC-AUC: 0.795 	> median: 0.673, mean: 0.656
****************************************************************
CPU times: user 3.82 s, sys: 223 ms, total: 4.05 s
Wall time: 1min 9s


In [35]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'gamma':  np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 1325, with 43 actives.
No. of molecules in test set: 500, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.915
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.918
- Best hyperparameters {'C': 100.0, 'gamma': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.644 	> median: 0.529, mean: 0.517
> Test best conf. ROC-AUC: 0.847 	> median: 0.739, mean: 0.720
****************************************************************
CPU times: user 3.98 s, sys: 189 ms, total: 4.17 s
Wall time: 1min 4s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'> GS: Logistic Regression </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [36]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: DUD; Test: DUD; split: random


/home/joel/anaconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 1368, with 43 actives.
No. of molecules in test set: 457, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.950
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.978
- Best hyperparameters {'C': 100.0, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.651 	> median: 0.557, mean: 0.546
> Test best conf. ROC-AUC: 0.758 	> median: 0.646, mean: 0.635
****************************************************************
CPU times: user 2.17 s, sys: 614 ms, total: 2.78 s
Wall time: 18.4 s


In [37]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: DUD; Test: DUD; split: scaffold


/home/joel/anaconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 1325, with 43 actives.
No. of molecules in test set: 500, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.914
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.895
- Best hyperparameters {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.644 	> median: 0.529, mean: 0.517
> Test best conf. ROC-AUC: 0.847 	> median: 0.739, mean: 0.720
****************************************************************
CPU times: user 2.08 s, sys: 558 ms, total: 2.64 s
Wall time: 9.49 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: K-Neighbors Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [38]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

kNN => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 1368, with 43 actives.
No. of molecules in test set: 457, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.841
- Train ROC-AUC:  	0.941
- Test ROC-AUC:   	0.802
- Best hyperparameters {'n_neighbors': 25, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.670 	> median: 0.575, mean: 0.560
> Test best conf. ROC-AUC: 0.729 	> median: 0.601, mean: 0.591
****************************************************************
CPU times: user 3.01 s, sys: 44.8 ms, total: 3.06 s
Wall time: 7.81 s


In [39]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = 'kNN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [25, 55, 125, 225], 
               'p': [1, 2]
             }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

kNN => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 1325, with 43 actives.
No. of molecules in test set: 500, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.815
- Train ROC-AUC:  	0.937
- Test ROC-AUC:   	0.880
- Best hyperparameters {'n_neighbors': 25, 'p': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.644 	> median: 0.529, mean: 0.517
> Test best conf. ROC-AUC: 0.847 	> median: 0.739, mean: 0.720
****************************************************************
CPU times: user 2.98 s, sys: 82.8 ms, total: 3.06 s
Wall time: 7.39 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [40]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 1368, with 43 actives.
No. of molecules in test set: 457, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.835
- Train ROC-AUC:  	0.914
- Test ROC-AUC:   	0.653
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.679 	> median: 0.593, mean: 0.580
> Test best conf. ROC-AUC: 0.702 	> median: 0.545, mean: 0.537
****************************************************************
CPU times: user 1.78 s, sys: 131 ms, total: 1.91 s
Wall time: 3.46 s


In [41]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2, 0.25, 0.3],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 1325, with 43 actives.
No. of molecules in test set: 500, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.741
- Train ROC-AUC:  	0.899
- Test ROC-AUC:   	0.722
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': 'log2', 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.644 	> median: 0.529, mean: 0.517
> Test best conf. ROC-AUC: 0.847 	> median: 0.739, mean: 0.720
****************************************************************
CPU times: user 1.69 s, sys: 112 ms, total: 1.8 s
Wall time: 3.3 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Bagging Classifier (k-NN as base estimator) </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [42]:
%%time
from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# RANDOM Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 55 µs, sys: 6 µs, total: 61 µs
Wall time: 70.1 µs


In [43]:
%%time
# from sklearn.ensemble import BaggingClassifier

estimator_name = 'BagClf-kNN'
knn = KNeighborsClassifier(n_neighbors=125, p=1, 
                           weights='distance')
estimator = BaggingClassifier(base_estimator=knn, 
                              n_jobs=6, oob_score=True)

hyperparams = {'n_estimators': [300]}

# SCAFFOLD Train test splitting
# split_and_gs(X, y, estimator, hyperparams, 
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 39 µs, sys: 4 µs, total: 43 µs
Wall time: 52 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest </h3>
<b>DUD  - Random and Stratified Scaffold Splitting</b>

In [44]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: DUD; Test: DUD; split: random
No. of molecules in train set: 1368, with 43 actives.
No. of molecules in test set: 457, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.864
- Train ROC-AUC:  	0.928
- Test ROC-AUC:   	0.813
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.1, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.679 	> median: 0.576, mean: 0.559
> Test best conf. ROC-AUC: 0.779 	> median: 0.601, mean: 0.597
****************************************************************
CPU times: user 3.06 s, sys: 75 ms, total: 3.13 s
Wall time: 27 s


In [45]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [2,  5],
             'min_samples_leaf': [0.1, 0.2],
             'max_features': ['log2', 'sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: DUD; Test: DUD; split: scaffold
No. of molecules in train set: 1325, with 43 actives.
No. of molecules in test set: 500, with 15 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.805
- Train ROC-AUC:  	0.920
- Test ROC-AUC:   	0.894
- Best hyperparameters {'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 0.1, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.644 	> median: 0.529, mean: 0.517
> Test best conf. ROC-AUC: 0.847 	> median: 0.739, mean: 0.720
****************************************************************
CPU times: user 2.44 s, sys: 85.2 ms, total: 2.52 s
Wall time: 23.5 s


***

<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Train-Test with different libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

##### Here, Random or Scaffold Splitting are not used

In [46]:
# Train DEKOIS, test DUD
@capture_GS_results(results_dict)
def split_by_library_and_gs(train_name, test_name, estimator_name,
                            X, y, lib_train_name, lib_test_name, **kwargs):
    '''Train-Test "split" by library, and run Grid Search. Splits the main dataframe by library
    using different molecular libraries for Training and Testing.'''
    splitting='by_library'
    
    X_train = X.loc[lib_train_name]
    y_train = y.loc[lib_train_name]

    X_test = X.loc[lib_test_name]
    y_test = y.loc[lib_test_name]

    # Verbose
    print(f'{estimator_name} => Train: {train_name}; Test: {test_name}; split: {splitting}')
    
    # Function to run Grid Search
    #----------------------------
    gs_results = run_grid_search(estimator, 
                    X_train, y_train, X_test, y_test, 
                    hyperparams = hyperparams,  **kwargs)

    # Function to extract ROC results from DkSc values 
    #-------------------------------------------------
    dksc_results = get_roc_auc_DkSc(X_train, y_train, X_test, y_test)
    
    # Return both list of results to captured by the decorador function
    return [train_name, test_name, estimator_name, splitting] + gs_results + dksc_results

In [47]:
# **********************************************
# Train and Test sets from diferent libraries
# **********************************************
X = X_merged_dksc
y = y_true_merged

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM  </h3>
<b>Train and Test with different libraries</b>

In [48]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6)}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LinearSVC => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1239, with 40 actives.
No. of molecules in test set: 1825, with 58 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.875
- Train ROC-AUC:  	0.976
- Test ROC-AUC:   	0.730
- Best hyperparameters {'C': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.790 	> median: 0.696, mean: 0.695
> Test best conf. ROC-AUC: 0.672 	> median: 0.584, mean: 0.568
****************************************************************
CPU times: user 2.11 s, sys: 30.2 ms, total: 2.14 s
Wall time: 7.11 s


In [49]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LinearSVC => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 1825, with 58 actives.
No. of molecules in test set: 1239, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.924
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.710
- Best hyperparameters {'C': 1.0}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.672 	> median: 0.584, mean: 0.568
> Test best conf. ROC-AUC: 0.790 	> median: 0.696, mean: 0.695
****************************************************************
CPU times: user 2.63 s, sys: 60.2 ms, total: 2.69 s
Wall time: 12.6 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM </h3>
<b>Train and Test with different libraries</b>

In [50]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

estimator_name = 'rbfSVC'
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'gamma':  np.geomspace(1e-8, 1e4, 7)}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

rbfSVC => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1239, with 40 actives.
No. of molecules in test set: 1825, with 58 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.873
- Train ROC-AUC:  	0.976
- Test ROC-AUC:   	0.730
- Best hyperparameters {'C': 100.0, 'gamma': 1e-06}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.790 	> median: 0.696, mean: 0.695
> Test best conf. ROC-AUC: 0.672 	> median: 0.584, mean: 0.568
****************************************************************
CPU times: user 4.12 s, sys: 262 ms, total: 4.38 s
Wall time: 1min 25s


In [51]:
%%time
estimator = SVC(kernel = 'rbf', probability=True)

estimator_name = 'rbfSVC'
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'gamma':  np.geomspace(1e-8, 1e4, 7)}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

rbfSVC => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 1825, with 58 actives.
No. of molecules in test set: 1239, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.934
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.653
- Best hyperparameters {'C': 100.0, 'gamma': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.672 	> median: 0.584, mean: 0.568
> Test best conf. ROC-AUC: 0.790 	> median: 0.696, mean: 0.695
****************************************************************
CPU times: user 5.04 s, sys: 267 ms, total: 5.31 s
Wall time: 2min 51s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Train and Test with different libraries</b>

In [52]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6),
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LogReg => Train: DEKOIS; Test: DUD; split: by_library


/home/joel/anaconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 1239, with 40 actives.
No. of molecules in test set: 1825, with 58 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.874
- Train ROC-AUC:  	0.958
- Test ROC-AUC:   	0.722
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.790 	> median: 0.696, mean: 0.695
> Test best conf. ROC-AUC: 0.672 	> median: 0.584, mean: 0.568
****************************************************************
CPU times: user 2.24 s, sys: 558 ms, total: 2.8 s
Wall time: 13.2 s


In [53]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

LogReg => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 1825, with 58 actives.
No. of molecules in test set: 1239, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.930
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.718
- Best hyperparameters {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.672 	> median: 0.584, mean: 0.568
> Test best conf. ROC-AUC: 0.790 	> median: 0.696, mean: 0.695
****************************************************************
CPU times: user 2.18 s, sys: 478 ms, total: 2.66 s
Wall time: 17.8 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS:Random Forest  </h3>
<b>Train and Test with different libraries</b>

In [54]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DEKOIS'
test_lib = 'DUD'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

RandForest => Train: DEKOIS; Test: DUD; split: by_library
No. of molecules in train set: 1239, with 40 actives.
No. of molecules in test set: 1825, with 58 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.776
- Train ROC-AUC:  	0.874
- Test ROC-AUC:   	0.708
- Best hyperparameters {'max_depth': 7, 'max_features': 'sqrt', 'min_samples_leaf': 0.05, 'min_samples_split': 0.2, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.790 	> median: 0.696, mean: 0.695
> Test best conf. ROC-AUC: 0.672 	> median: 0.584, mean: 0.568
****************************************************************
CPU times: user 3.81 s, sys: 50.6 ms, total: 3.86 s
Wall time: 50 s


In [55]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 500],
             'max_depth': [3,  5, 7],
             'min_samples_leaf': [0.03, 0.05],
               'min_samples_split': [0.2,  0.3],
             'max_features': ['sqrt']
            }

# Train DEKOIS, test DUD
train_lib = 'DUD'
test_lib = 'DEKOIS'
split_by_library_and_gs(
    train_lib, test_lib, estimator_name,
    X, y, train_lib, test_lib)

RandForest => Train: DUD; Test: DEKOIS; split: by_library
No. of molecules in train set: 1825, with 58 actives.
No. of molecules in test set: 1239, with 40 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.821
- Train ROC-AUC:  	0.915
- Test ROC-AUC:   	0.665
- Best hyperparameters {'max_depth': 5, 'max_features': 'sqrt', 'min_samples_leaf': 0.03, 'min_samples_split': 0.3, 'n_estimators': 500}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.672 	> median: 0.584, mean: 0.568
> Test best conf. ROC-AUC: 0.790 	> median: 0.696, mean: 0.695
****************************************************************
CPU times: user 4.21 s, sys: 91.9 ms, total: 4.3 s
Wall time: 1min 4s


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries 
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD, CSAR and COCRYS are  treated as one unique library

In [14]:
# Train and test over 
X = X_merged_dksc
y = y_true_merged
library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']
(scaffold_series.index == X.index).sum()

3466

<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [57]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LinearSVC => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.878
- Train ROC-AUC:  	0.944
- Test ROC-AUC:   	0.875
- Best hyperparameters {'C': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.680 	> median: 0.627, mean: 0.621
> Test best conf. ROC-AUC: 0.646 	> median: 0.577, mean: 0.574
****************************************************************
CPU times: user 7.18 s, sys: 124 ms, total: 7.31 s
Wall time: 1min 33s


In [58]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LinearSVC => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 2288, with 311 actives.
No. of molecules in test set: 1178, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.854
- Train ROC-AUC:  	0.963
- Test ROC-AUC:   	0.794
- Best hyperparameters {'C': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.661 	> median: 0.609, mean: 0.603
> Test best conf. ROC-AUC: 0.745 	> median: 0.646, mean: 0.643
****************************************************************
CPU times: user 6.53 s, sys: 68.2 ms, total: 6.6 s
Wall time: 49.3 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [59]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'gamma':  np.geomspace(1e-8, 1e2, 6)}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

rbfSVC => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.923
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.923
- Best hyperparameters {'C': 100.0, 'gamma': 0.01}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.659 	> median: 0.597, mean: 0.593
> Test best conf. ROC-AUC: 0.725 	> median: 0.663, mean: 0.656
****************************************************************
CPU times: user 19.8 s, sys: 366 ms, total: 20.2 s
Wall time: 8min 11s


In [60]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'gamma': np.geomspace(1e-8, 1e2, 6)}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

rbfSVC => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 2288, with 311 actives.
No. of molecules in test set: 1178, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.873
- Train ROC-AUC:  	0.982
- Test ROC-AUC:   	0.800
- Best hyperparameters {'C': 100.0, 'gamma': 0.0001}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.661 	> median: 0.609, mean: 0.603
> Test best conf. ROC-AUC: 0.745 	> median: 0.646, mean: 0.643
****************************************************************
CPU times: user 8.53 s, sys: 240 ms, total: 8.77 s
Wall time: 6min 16s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [61]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=150)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

LogReg => Train: Merged; Test: Merged; split: random


/home/joel/anaconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.889
- Train ROC-AUC:  	0.940
- Test ROC-AUC:   	0.871
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.670 	> median: 0.615, mean: 0.610
> Test best conf. ROC-AUC: 0.674 	> median: 0.614, mean: 0.607
****************************************************************
CPU times: user 3.42 s, sys: 863 ms, total: 4.28 s
Wall time: 4min 51s


In [62]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression(max_iter=200)
hyperparams = {'C':  np.geomspace(1e-8, 1e2, 6), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

LogReg => Train: Merged; Test: Merged; split: scaffold


/home/joel/anaconda3/envs/mds/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


No. of molecules in train set: 2288, with 311 actives.
No. of molecules in test set: 1178, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.860
- Train ROC-AUC:  	0.952
- Test ROC-AUC:   	0.815
- Best hyperparameters {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.661 	> median: 0.609, mean: 0.603
> Test best conf. ROC-AUC: 0.745 	> median: 0.646, mean: 0.643
****************************************************************
CPU times: user 3.34 s, sys: 901 ms, total: 4.24 s
Wall time: 2min 12s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Decision Tree Classifier</h3>
<b>Merged Libraries</b>

In [63]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

DTree => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.728
- Train ROC-AUC:  	0.771
- Test ROC-AUC:   	0.771
- Best hyperparameters {'criterion': 'entropy', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 0.02, 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.667 	> median: 0.606, mean: 0.602
> Test best conf. ROC-AUC: 0.692 	> median: 0.638, mean: 0.632
****************************************************************
CPU times: user 2.83 s, sys: 154 ms, total: 2.99 s
Wall time: 7.8 s


In [64]:
%%time
from sklearn.tree import DecisionTreeClassifier

estimator_name = 'DTree'
estimator = DecisionTreeClassifier(splitter='best')
hyperparams = {'criterion': ['gini', 'entropy'], 
               'max_depth': [2, 3, 5],
               'min_samples_split': [0.2,  0.3],
               'min_samples_leaf': [0.02, 0.05, 0.1],
               'max_features': [None, 'sqrt', 'log2']}

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

DTree => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 2288, with 311 actives.
No. of molecules in test set: 1178, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.769
- Train ROC-AUC:  	0.826
- Test ROC-AUC:   	0.646
- Best hyperparameters {'criterion': 'gini', 'max_depth': 5, 'max_features': None, 'min_samples_leaf': 0.02, 'min_samples_split': 0.2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.661 	> median: 0.609, mean: 0.603
> Test best conf. ROC-AUC: 0.745 	> median: 0.646, mean: 0.643
****************************************************************
CPU times: user 2.93 s, sys: 126 ms, total: 3.05 s
Wall time: 7.51 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: KKN - 1-NN as reference estimator </h3>
<b>Merged Libraries</b>

In [26]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = '1-NN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [1], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

1-NN => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.781
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.776
- Best hyperparameters {'n_neighbors': 1, 'p': 2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.669 	> median: 0.615, mean: 0.611
> Test best conf. ROC-AUC: 0.677 	> median: 0.612, mean: 0.604
****************************************************************
CPU times: user 2.93 s, sys: 79.9 ms, total: 3.01 s
Wall time: 6.4 s


In [25]:
%%time
from sklearn.neighbors import KNeighborsClassifier 

estimator_name = '1-NN'
estimator = KNeighborsClassifier()
hyperparams = {'n_neighbors': [1], 
               'p': [1, 2]
             }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=scaffold_series)

1-NN => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.815
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.794
- Best hyperparameters {'n_neighbors': 1, 'p': 2}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.682 	> median: 0.626, mean: 0.620
> Test best conf. ROC-AUC: 0.640 	> median: 0.578, mean: 0.576
****************************************************************
CPU times: user 2.7 s, sys: 99.6 ms, total: 2.8 s
Wall time: 9.06 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Random Forest </h3>
<b>Merged Libraries</b>

In [67]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None)

RandForest => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.780
- Train ROC-AUC:  	0.821
- Test ROC-AUC:   	0.776
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.05, 'min_samples_split': 0.1, 'n_estimators': 400}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.663 	> median: 0.609, mean: 0.603
> Test best conf. ROC-AUC: 0.705 	> median: 0.633, mean: 0.628
****************************************************************
CPU times: user 3.89 s, sys: 87.6 ms, total: 3.97 s
Wall time: 39.1 s


In [68]:
%%time
from sklearn.ensemble import RandomForestClassifier

estimator_name = 'RandForest'
estimator = RandomForestClassifier()
hyperparams = {'n_estimators': [300, 400],
               'max_depth': [2, 3],
               'min_samples_split': [0.1,  0.3],
               'min_samples_leaf': [0.02, 0.05],
               'max_features': ['sqrt']
            }

# SCAFFOLD Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series)

RandForest => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 2288, with 311 actives.
No. of molecules in test set: 1178, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.780
- Train ROC-AUC:  	0.839
- Test ROC-AUC:   	0.690
- Best hyperparameters {'max_depth': 3, 'max_features': 'sqrt', 'min_samples_leaf': 0.05, 'min_samples_split': 0.1, 'n_estimators': 300}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.661 	> median: 0.609, mean: 0.603
> Test best conf. ROC-AUC: 0.745 	> median: 0.646, mean: 0.643
****************************************************************
CPU times: user 3.23 s, sys: 97.1 ms, total: 3.32 s
Wall time: 33.2 s


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: X Gradient Boosting </h3>
<b>Merged Libraries</b>

In [20]:
%%time

from xgboost import XGBClassifier

estimator_name = 'XGB_tree'
estimator = XGBClassifier()
hyperparams = {'n_estimators': [200, 300],
               'max_depth': [2, 3, 10, 20],
               'learning_rate': [0.05, 0.1],
               'gamma': [0.01, 0.1, 0.5, 1],
               'alpha': [0.01, 0.1, 0.5, 1],
               'subsample': [0.3, 0.5],
               'colsample_bytree': [0.3, 0.5, 1]
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='random', test_size=0.25, 
             scaffold_series=None,
             # RandomizedGS
            randomGS=True, n_iter=50)

XGB_tree => Train: Merged; Test: Merged; split: random
No. of molecules in train set: 2599, with 311 actives.
No. of molecules in test set: 867, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.905
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.912
- Best hyperparameters {'subsample': 0.3, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0.5, 'colsample_bytree': 1, 'alpha': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.669 	> median: 0.608, mean: 0.603
> Test best conf. ROC-AUC: 0.691 	> median: 0.631, mean: 0.626
****************************************************************
CPU times: user 26.5 s, sys: 6.29 s, total: 32.8 s
Wall time: 6min 15s


In [19]:
%%time

from xgboost import XGBClassifier

estimator_name = 'XGB_tree'
estimator = XGBClassifier()
hyperparams = {'n_estimators': [200, 300],
               'max_depth': [2, 3, 10],
               'learning_rate': [0.05, 0.1],
               'gamma': [0.01, 0.1, 0.5, 1],
               'alpha': [0.01, 0.1, 0.5, 1],
               'subsample': [0.3, 0.5],
               'colsample_bytree': [0.3, 0.5, 1]
            }

# RANDOM Train test splitting
split_and_gs(library, library, estimator_name,
             X, y, estimator, hyperparams,
             splitting='scaffold', test_size=0.25, 
             scaffold_series=scaffold_series,
             # RandomizedGS
             randomGS=True, n_iter=50)

XGB_tree => Train: Merged; Test: Merged; split: scaffold
No. of molecules in train set: 2288, with 311 actives.
No. of molecules in test set: 1178, with 104 actives.

********** GRID SEARCH RESULTS **********
- Mean CV ROC-AUC:	0.863
- Train ROC-AUC:  	1.000
- Test ROC-AUC:   	0.775
- Best hyperparameters {'subsample': 0.3, 'n_estimators': 200, 'max_depth': 10, 'learning_rate': 0.1, 'gamma': 0.1, 'colsample_bytree': 0.3, 'alpha': 1}
******************************************

***** Best Conformation's ROC-AUC using docking scores *****
> Train best conf. ROC-AUC: 0.661 	> median: 0.609, mean: 0.603
> Test best conf. ROC-AUC: 0.745 	> median: 0.646, mean: 0.643
****************************************************************
CPU times: user 11.6 s, sys: 3.48 s, total: 15.1 s
Wall time: 4min 23s


## Get, Format, and Save the Results

In [21]:
### Save Results
row_names = ['Train_set', 'Test_set', 'Model name', 'Split', 
             'N_actives_train', 'N_actives_test', 'N_mols_train', 'Num_mols_test',
            'Mean-CV-ROC', 'ROC-AUC_train', 'ROC-AUC_test', 'best_params',
            'DkS_max_ROC_train',  'DkSc_med_ROC_train', 'DkSc_mean_ROC_train', 
            'DkS_max_ROC_test',  'DkSc_med_ROC_test', 'DkSc_mean_ROC_test'

# Load or save
save=False
filename='./GRID_SEARCH_CV_results.csv'
if save:
    results_df = pd.DataFrame(results_dict, index=row_names).T.reset_index().drop('index', axis=1)
    results_df.to_csv(filename)
else:
    results_df = pd.read_csv(filename)
    
results_df

,Unnamed: 0,Train_set,Test_set,Model name,Split,N_actives_train,N_actives_test,N_mols_train,Num_mols_test,Mean-CV-ROC,ROC-AUC_train,ROC-AUC_test,best_params,DkS_max_ROC_train,DkSc_med_ROC_train,DkSc_mean_ROC_train,DkS_max_ROC_test,DkSc_med_ROC_test,DkSc_mean_ROC_test
0,0,DEKOIS,DEKOIS,LinearSVC,random,929,30,310,10,0.857541,0.993215,0.838000,{'C': 0.0001},0.821913,0.708398,0.706996,0.835167,0.649167,0.656708
1,1,DEKOIS,DEKOIS,LinearSVC,scaffold,899,30,340,10,0.821941,0.967511,0.838788,{'C': 0.0001},0.828692,0.733381,0.731857,0.808333,0.595909,0.596055
2,2,DEKOIS,DEKOIS,rbfSVC,random,929,30,310,10,0.848584,0.976492,0.837667,"{'C': 100.0, 'gamma': 1e-06}",0.814349,0.706674,0.703665,0.815333,0.665500,0.668252
3,3,DEKOIS,DEKOIS,rbfSVC,scaffold,899,30,340,10,0.828257,0.955850,0.873030,"{'C': 100.0, 'gamma': 1e-08}",0.828692,0.733381,0.731857,0.808333,0.595909,0.596055
4,4,DEKOIS,DEKOIS,LogReg,random,929,30,310,10,0.871050,0.960030,0.836333,"{'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}",0.808862,0.687987,0.687810,0.829000,0.717750,0.716770
5,5,DEKOIS,DEKOIS,LogReg,scaffold,899,30,340,10,0.831961,0.934829,0.825152,"{'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}",0.828692,0.733381,0.731857,0.808333,0.595909,0.596055
6,6,DEKOIS,DEKOIS,kNN,random,929,30,310,10,0.772427,0.771098,0.731833,"{'n_neighbors': 225, 'p': 1}",0.778198,0.692362,0.692595,0.883500,0.702417,0.702298
7,7,DEKOIS,DEKOIS,kNN,scaffold,899,30,340,10,0.812489,0.827771,0.636364,"{'n_neighbors': 125, 'p': 2}",0.828692,0.733381,0.731857,0.808333,0.595909,0.596055
8,8,DEKOIS,DEKOIS,DTree,random,929,30,310,10,0.707495,0.761494,0.712000,"{'criterion': 'gini', 'max_depth': 3, 'max_fea...",0.800204,0.687060,0.686551,0.894667,0.721833,0.725884
9,9,DEKOIS,DEKOIS,DTree,scaffold,899,30,340,10,0.742715,0.912179,0.647273,"{'criterion': 'entropy', 'max_depth': 5, 'max_...",0.828692,0.733381,0.731857,0.808333,0.595909,0.596055


***
<h2 style='background-color: #F9E5AB; padding: 5px;'>
    Merged libraries: Shuffle *y* target values in the train set
</h2>
<div style='background-color: #FE8460; min-height: 5px'></div>

#### DEKOIS, DUD and COCRYS are  treated as one unique library

In [70]:
# Train and test over 
X = X_merged_dksc
# ***** Permutate y values *****
y = y_true_merged.sample(frac=1)

library = 'Merged'
scaffold_series = df_scff_murcko['scff_generic']


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Linear SVM </h3>
<b>Merged Libraries</b>

In [71]:
%%time
from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 347 µs, sys: 26 µs, total: 373 µs
Wall time: 325 µs


In [72]:
%%time
# from sklearn.svm import SVC

estimator_name = 'LinearSVC'
estimator = SVC(kernel = 'linear', probability=True)
hyperparams = {'C': np.geomspace(1e-15, 1e-9, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 340 µs, sys: 25 µs, total: 365 µs
Wall time: 320 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Radial Basis Function SVM</h3>
<b>Merged Libraries</b>

In [73]:
%%time
from sklearn.svm import SVC

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 439 µs, sys: 32 µs, total: 471 µs
Wall time: 390 µs


In [74]:
%%time

estimator_name = 'rbfSVC'
estimator = SVC(kernel = 'rbf', probability=True)
hyperparams = {'C': np.geomspace(1e0, 1e2, 3), 
               'gamma': np.geomspace(1e-4, 1e0, 3)}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 524 µs, sys: 0 ns, total: 524 µs
Wall time: 424 µs


<h3 style='color: #F84122; padding: 0px;; margin: 0px'>GS: Logistic Regression</h3>
<b>Merged Libraries</b>

In [75]:
%%time
from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e4, 3), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# RANDOM Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='random', test_size=0.25, 
#              scaffold_series=None)

CPU times: user 294 µs, sys: 22 µs, total: 316 µs
Wall time: 275 µs


In [76]:
%%time
# from sklearn.linear_model import LogisticRegression

estimator_name = 'LogReg'
estimator = LogisticRegression()
hyperparams = {'C': np.geomspace(1e-4, 1e5, 4), 
               'penalty': ['l1', 'l2'], 
               'solver': ['lbfgs', 'liblinear']}

# SCAFFOLD Train test splitting
# split_and_gs(library, library, estimator_name,
#              X, y, estimator, hyperparams,
#              splitting='scaffold', test_size=0.25, 
#              scaffold_series=scaffold_series)

CPU times: user 312 µs, sys: 23 µs, total: 335 µs
Wall time: 290 µs
